In [1]:
import pandas as pd

In [2]:
chennels_df = pd.read_csv('data/chennels_df.csv')
chennels_df['chennel@'] = chennels_df['hrefs'].apply(lambda x: x[26:])

In [3]:
from telethon import TelegramClient, events, sync
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.functions.channels import JoinChannelRequest,LeaveChannelRequest

# These example values won't work. You must get your own api_id and
# api_hash from https://my.telegram.org, under API Development.
api_id = 1522790
api_hash = '00008593fd02991e904e01df2abe2310'

client = TelegramClient('session_name', api_id, api_hash)
client.start()

In [4]:
def parse_n_messages(channel_name, n, client):
    channel = client.get_entity(channel_name)
    posts = client(GetHistoryRequest(
    peer=channel,
    limit=n,
    offset_date=None,
    offset_id=0,
    max_id=0,
    min_id=0,
    add_offset=0,
    hash=0))

    post_texts = [post.message for post in posts.messages]
    chennel_posts_df = pd.DataFrame(zip(post_texts,posts.messages), columns = ['text', 'message'])
    return chennel_posts_df

In [8]:
posts_df = pd.DataFrame()


In [9]:
classes = ['business', 'entertainment', 'economics', 'crypto',
       'education', 'music', 'language', 'tech', 'psychology',
       'marketing', 'career',
       'health', 'other']

chennels_df_to_parse = chennels_df[chennels_df['class'].isin(classes)]

In [ ]:
%%time
for i, (href, name, class_name, channel_name) in chennels_df_to_parse.iterrows():
    client(JoinChannelRequest(channel_name))
    
    chennel_posts_df = parse_n_messages(channel_name, 150, client)
    chennel_posts_df['class'] = class_name
    chennel_posts_df['channel'] = channel_name

    
    client(LeaveChannelRequest(channel_name))
    
    posts_df = pd.concat([posts_df,chennel_posts_df], axis = 0)


In [ ]:
posts_df.to_csv('data/posts_df.csv', index = False)

In [ ]:
posts_df